In [1]:
import os, sys 
%pwd

'/Users/somtoachi-kanu/End to End Folder/IBM anti launder transaction/IBM-anti-laundering-transaction/notebooks'

In [2]:
os.chdir("../")
%pwd

'/Users/somtoachi-kanu/End to End Folder/IBM anti launder transaction/IBM-anti-laundering-transaction'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    gamma: int
    max_depth: int
    colsample_bytree: int
    reg_alpha: int
    reg_lambda: int
    subsample: int
    target_column: str

In [4]:
from IBM_anti_launder_transaction.constants import * 
from IBM_anti_launder_transaction.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            gamma = params.gamma,
            max_depth = params.max_depth,
            colsample_bytree = params.colsample_bytree,
            reg_alpha = params.reg_alpha,
            reg_lambda = params.reg_lambda,
            subsample = params.subsample,
            target_column = schema.name
            
        )

        return model_trainer_config

In [10]:
import pandas as pd
import os
from IBM_anti_launder_transaction import logger
import joblib
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/somtoachi-kanu/End to End Folder/IBM anti launder transaction/IBM-anti-laundering-transaction/venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <6749C431-FD87-3912-A7CF-11CB9194D072> /Users/somtoachi-kanu/End to End Folder/IBM anti launder transaction/IBM-anti-laundering-transaction/venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no 

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        xgb_model = XGBClassifier()
        xgb_model.fit(train_x, train_y)

        joblib.dump(xgb_model, os.path.join(self.config.root_dir, self.config.model_name))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e